In [7]:
import pandas as pd

#use the mapping dictionary imported from the teams.csv file to change all of the full team names to the abbreviation

abbreviations_df = pd.read_csv('teams.csv')
mapping_dict = dict(zip(abbreviations_df['Team'], abbreviations_df['Abbrv']))

# mapping function
def update_and_overwrite_file(file_path, mapping_dict):
    df = pd.read_csv(file_path)
    df['Team'] = df['Team'].map(mapping_dict)
    df.to_csv(file_path, index=False)  # Overwrite the original file

file_paths = ['C:/Users/luken/Desktop/NHL_Model/Raw_data/20_21_games.csv', 'C:/Users/luken/Desktop/NHL_Model/Raw_data/21_22_games.csv', 'C:/Users/luken/Desktop/NHL_Model/Raw_data/22_23_games.csv']

#loop through the files and apply the function
for file_path in file_paths:
    update_and_overwrite_file(file_path, mapping_dict)

In [8]:
import pandas as pd
import re

#use the mappping dictionary to change all the shortenned team names to the abbreviation
abbreviations_df = pd.read_csv('teams.csv')
mapping_dict = dict(zip(abbreviations_df['Name'], abbreviations_df['Abbrv']))

#the team names are within the string in the "game" column in the file so to change them we need to deconstruct the string into the parts, rename the teams with the mapping dictionary and then reconstruct the string
def replace_team_names(game_string, mapping_dict):
    pattern = r' (\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
    match = re.match(pattern, game_string)

    if match:
        date, team1, score1, team2, score2 = match.groups()
        team1_abbr = mapping_dict.get(team1.strip(), team1)  # Fallback to original if not found
        team2_abbr = mapping_dict.get(team2.strip(), team2)  # Fallback to original if not found

        return f"{date} - {team1_abbr} {score1}, {team2_abbr} {score2}"
    else:
        return game_string 

def update_and_overwrite_file(file_path, mapping_dict):
    df = pd.read_csv(file_path)
    df['Game'] = df['Game'].apply(lambda x: replace_team_names(x, mapping_dict))
    df.to_csv(file_path, index=False)

file_paths = ['C:/Users/luken/Desktop/NHL_Model/Raw_data/20_21_games.csv', 'C:/Users/luken/Desktop/NHL_Model/Raw_data/21_22_games.csv', 'C:/Users/luken/Desktop/NHL_Model/Raw_data/22_23_games.csv']

#loop through again
for file_path in file_paths:
    update_and_overwrite_file(file_path, mapping_dict)

In [9]:
import pandas as pd
import os
input_dir = "C:/Users/luken/Desktop/NHL_Model/Raw_data/" 
output_dir = "C:/Users/luken/Desktop/NHL_Model/Selected_data"  
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#drop the unwanted columns from the files
#every stat has a for column, agaisnt column, and a for% column, so we willl drop all of the columns that have a % in the name and jsut use the for and agaisnt counts to reduce redundancy in the data


csv_files = [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f)) and f.endswith('.csv')]

for file in csv_files:
    input_file_path = os.path.join(input_dir, file)
    output_file_path = os.path.join(output_dir, file)
    
    df = pd.read_csv(input_file_path)
    
    cols_to_drop = [col for col in df.columns if '%' in col]
    
    #dropping other unnecceasry columns (no way to get attendance before the games so we cant use it to predict future games)
    df.drop(columns=cols_to_drop, inplace=True)
    df.drop('Unnamed: 2', axis=1, inplace=True)
    df.drop('Attendance', axis=1, inplace=True)
    df.drop('TOI', axis=1, inplace=True)

    df.to_csv(output_file_path, index=False)
    


Processed and saved 20_21_games.csv to C:/Users/luken/Desktop/NHL_Model/Selected_data
Processed and saved 21_22_games.csv to C:/Users/luken/Desktop/NHL_Model/Selected_data
Processed and saved 22_23_games.csv to C:/Users/luken/Desktop/NHL_Model/Selected_data


In [10]:
import os
import pandas as pd
input_directory = "C:/Users/luken/Desktop/NHL_Model/Selected_data/"
base_output_directory = "C:/Users/luken/Desktop/NHL_Model/Sorted_data"

#each file represents all of the games from the past year, with every unique game occuring twice, once for the home team and then a second time from the perspective fo the away team
#the value in the "team" column shows which team's persepctive the stats are from, so we can sort each csv file into a unique file for each team from each year

#iterate through the files and create new files for each team's games for each year
for file_name in os.listdir(input_directory):
    if file_name.endswith(".csv"):
        year = file_name.split('_games.csv')[0]

        year_directory = os.path.join(base_output_directory, year)
        if not os.path.exists(year_directory):
            os.makedirs(year_directory)

        file_path = os.path.join(input_directory, file_name)
        df = pd.read_csv(file_path)
        #using this patern from the stirng in the "game" column we can get the home and away teams and scores, and then use the scores to determine the winner of the game
        pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
        df[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = df['Game'].str.extract(pattern)
        df['HomeResult'] = 'Draw'
        df.loc[df['HomeScore'] > df['AwayScore'], 'HomeResult'] = 'Won'
        df.loc[df['HomeScore'] < df['AwayScore'], 'HomeResult'] = 'Lost'
        


        team_subsets = {team: df[df['Team'] == team] for team in df['Team'].unique()}

        for team, subset in team_subsets.items():
            team_file_name = f"{team}.csv"
            team_file_path = os.path.join(year_directory, team_file_name)
            subset.to_csv(team_file_path, index=False)



CSV files organized by year and saved in C:/Users/luken/Desktop/NHL_Model/Sorted_data


In [11]:
#loop through the files for each team in each year to calculate the number of days of rest each team had going into that game by taking the difference between the number of days between each game (time diff) minus 1 (days of rest)

root_dir = 'sorted_data/'

subfolders = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]

for subfolder in subfolders:
    dir_path = os.path.join(root_dir, subfolder)
    
   
    all_files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f)) and f.endswith('.csv')]

    for file in all_files:
        file_path = os.path.join(dir_path, file)
        df = pd.read_csv(file_path)
        if 'Date' in df.columns:
            date_col = 'Date'
        else:
            print(f"Cannot find date column in file {file} inside {subfolder}. Skipping...")
            continue
        df[date_col] = pd.to_datetime(df[date_col])
        df['time_diff'] = df[date_col].diff()
        df['days_of_rest'] = df['time_diff'].dt.days - 1
        df.drop('time_diff', axis=1, inplace=True)
        df.to_csv(file_path, index=False)


Processed ANA.csv inside 20_21
Processed ARI.csv inside 20_21
Processed BOS.csv inside 20_21
Processed BUF.csv inside 20_21
Processed CAR.csv inside 20_21
Processed CBJ.csv inside 20_21
Processed CGY.csv inside 20_21
Processed CHI.csv inside 20_21
Processed COL.csv inside 20_21
Processed DAL.csv inside 20_21
Processed DET.csv inside 20_21
Processed EDM.csv inside 20_21
Processed FLA.csv inside 20_21
Processed LA.csv inside 20_21
Processed MIN.csv inside 20_21
Processed MTL.csv inside 20_21
Processed NJ.csv inside 20_21
Processed NSH.csv inside 20_21
Processed NYI.csv inside 20_21
Processed NYR.csv inside 20_21
Processed OTT.csv inside 20_21
Processed PHI.csv inside 20_21
Processed PIT.csv inside 20_21
Processed SJ.csv inside 20_21
Processed STL.csv inside 20_21
Processed TB.csv inside 20_21
Processed TOR.csv inside 20_21
Processed VAN.csv inside 20_21
Processed VGK.csv inside 20_21
Processed WPG.csv inside 20_21
Processed WSH.csv inside 20_21
Processed ANA.csv inside 21_22
Processed AR

In [12]:
import os
import pandas as pd
#create a function to calculate the rolling average of the 10 games prior to each game to get the rolling average of stats going into each game
def calculate_rolling_average(df, columns, window=10):
    rolling_df = df[columns].rolling(window=window, min_periods=10).mean().shift(1)
    return rolling_df

base_path = 'sorted_data/'
#create a new folder to put the new processed data into
average_data_path = 'average_data/'
os.makedirs(average_data_path, exist_ok=True)

for year_folder in os.listdir(base_path):
    year_path = os.path.join(base_path, year_folder)

    if os.path.isdir(year_path):
        year_avg_folder_path = os.path.join(average_data_path, f'avg_{year_folder}')
        os.makedirs(year_avg_folder_path, exist_ok=True)

        for team_file in os.listdir(year_path):
            if team_file.endswith('.csv'):
                team_path = os.path.join(year_path, team_file)
                df = pd.read_csv(team_path)
                #select the columns we dont want to average (all non numerical, score and days of rest)
                selected_columns = ['Game', 'Team', 'Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult', 'days_of_rest']
                other_columns = df.columns.difference(selected_columns)
                rolling_df = calculate_rolling_average(df, other_columns)
                #re-combine the non-averaged columns with the averaged columns
                combined_df = pd.concat([df[selected_columns], rolling_df], axis=1)

                processed_file_path = os.path.join(year_avg_folder_path, f'{team_file}')
                combined_df.to_csv(processed_file_path, index=False)

In [13]:
#in order to build the data set for creating the training set, we need to take a list of each game from the original files
directory_path = 'Raw_data'

# create the initial empty dataframe to add the games to 
master_df = pd.DataFrame()

#loop through all of the files in the raw data folder and add the game into the the master dataframe
for file_name in os.listdir(directory_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(directory_path, file_name)
        df = pd.read_csv(file_path)
        if 'Game' in df.columns:
            master_df = pd.concat([master_df, df[['Game']]])

#extract the information from the string in the game column
pattern = r'(\d{4}-\d{2}-\d{2}) - ([\w\s]+) (\d+), ([\w\s]+) (\d+)'
master_df[['Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore']] = master_df['Game'].str.extract(pattern)
master_df['HomeResult'] = 'Draw'
master_df.loc[master_df['HomeScore'] > master_df['AwayScore'], 'HomeResult'] = 'Won'
master_df.loc[master_df['HomeScore'] < master_df['AwayScore'], 'HomeResult'] = 'Lost'
filt_df = master_df.iloc[1::2]
filt_df.head(20)


,Game,Date,AwayTeam,AwayScore,HomeTeam,HomeScore,HomeResult
1,"2021-01-13 - PIT 3, PHI 6",2021-01-13,PIT,3,PHI,6,Won
3,"2021-01-13 - CHI 1, TB 5",2021-01-13,CHI,1,TB,5,Won
5,"2021-01-13 - MTL 4, TOR 5",2021-01-13,MTL,4,TOR,5,Won
7,"2021-01-13 - VAN 5, EDM 3",2021-01-13,VAN,5,EDM,3,Lost
9,"2021-01-13 - STL 4, COL 1",2021-01-13,STL,4,COL,1,Lost
11,"2021-01-14 - WSH 6, BUF 4",2021-01-14,WSH,6,BUF,4,Lost
13,"2021-01-14 - BOS 3, NJ 2",2021-01-14,BOS,3,NJ,2,Lost
15,"2021-01-14 - NYI 4, NYR 0",2021-01-14,NYI,4,NYR,0,Lost
17,"2021-01-14 - CAR 3, DET 0",2021-01-14,CAR,3,DET,0,Lost
19,"2021-01-14 - CBJ 1, NSH 3",2021-01-14,CBJ,1,NSH,3,Won


In [14]:
#combine the data from each year into one file for each team to then iterate through to pull the stats for each game
year_directories = ['average_data/avg_20_21/', 'average_data/avg_21_22/', 'average_data/avg_22_23/']

combined_directory = 'stats'
os.makedirs(combined_directory, exist_ok=True)


team_dfs = {}

for directory in year_directories:
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            team_name = filename.replace('_games.csv', '')
            file_path = os.path.join(directory, filename)

            df = pd.read_csv(file_path)
            cols_to_drop = ['Date','AwayTeam','AwayScore','HomeTeam','HomeScore','HomeResult']
            df.drop(columns=cols_to_drop, inplace=True)
            if team_name in team_dfs:
                team_dfs[team_name] = pd.concat([team_dfs[team_name], df])
            else:
                team_dfs[team_name] = df

for team_name, df in team_dfs.items():
    output_path = os.path.join(combined_directory, f'{team_name}')
    df.to_csv(output_path, index=False)

In [15]:
#take the master dataframe with all of the games and add the stats for the home team, adding a prefix of h to the stat to indicate is is from the home team, creating a new dataframe with all of the home stats
games_df = filt_df
#create the function to add the hoem team stats
def add_home_team_stats(row):
    team_name = row['HomeTeam']
    file_path = f'stats/{team_name}.csv'
    team_stats_df = pd.read_csv(file_path)
    game_row = team_stats_df[team_stats_df['Game'] == row['Game']]
    if not game_row.empty:
        team_stats = game_row.add_prefix('h_').iloc[0]
        team_stats.rename({'h_Game': 'Game'}, inplace=True)
        for col in team_stats.index:
            row[col] = team_stats[col]
    return row
#initiate the list of updated rows with the stats
updated_rows = []
#call the function on every row
for index, row in games_df.iterrows():
    row = add_home_team_stats(row) 
    updated_rows.append(row)
hgames_df = pd.DataFrame(updated_rows)


In [16]:
#repeat the previous step but for away team stats
games_df = filt_df

def add_away_team_stats(row):
    team_name = row['AwayTeam']
    file_path = f'stats/{team_name}.csv'
    team_stats_df = pd.read_csv(file_path)
    game_row = team_stats_df[team_stats_df['Game'] == row['Game']]
    if not game_row.empty:
        team_stats = game_row.add_prefix('a_').iloc[0]
        team_stats.rename({'a_Game': 'Game'}, inplace=True)
        for col in team_stats.index:
            row[col] = team_stats[col]
    return row

updated_rows = []
for index, row in games_df.iterrows():
    row = add_away_team_stats(row) 
    updated_rows.append(row)
agames_df = pd.DataFrame(updated_rows)


In [17]:
#merge the two dataframes to create one master dataframe with all the away and home team stats, then save the datraframe to the a csv file
import pandas as pd
common_columns = ['Game', 'Date', 'AwayTeam', 'AwayScore', 'HomeTeam', 'HomeScore', 'HomeResult']

merged_df = pd.merge(agames_df, hgames_df, on=common_columns, how='outer')
df2 = merged_df.dropna()
cols_to_drop = ['a_Team','h_Team']
df2.drop(columns=cols_to_drop, inplace=True)

df2.to_csv('training_set.csv', index=False)
#the training set is now stored in "training_set.csv"

C:\Users\luken\AppData\Local\Temp\ipykernel_6820\2273431247.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop(columns=cols_to_drop, inplace=True)
